# Internship Finn Sherry @ Sioux Mathware

---

# Bayesian grey-box system identification for thermal effects: Biased Coins and MAB
In this notebook, we will become familiar with Bayesian inference. First, we will try to identify the bias of a coin. Then, we will tackle the Multi-Armed Bandit (MAB) problem.

Last update: 19-07-2022

$\renewcommand{\vec}[1]{\boldsymbol{\mathrm{#1}}}$
$\newcommand{\covec}[1]{\hat{\vec{#1}}}$
$\newcommand{\mat}[1]{\boldsymbol{\mathrm{#1}}}$
$\newcommand{\inv}[1]{#1^{-1}}$
$\newcommand{\given}{\, \vert \,}$
$\newcommand{\haslaw}{\sim}$
$\newcommand{\problaw}[1]{p(#1)}$
$\newcommand{\Expectation}{\mathbb{E}}$
$\newcommand{\Variance}{\mathbb{V}}$
$\newcommand{\Geometric}{\textrm{Geom}}$
$\newcommand{\NegBin}{\textrm{NB}}$
$\newcommand{\Poisson}{\textrm{Pois}}$
$\newcommand{\Bernoulli}{\textrm{Bern}}$
$\newcommand{\Uniform}{\textrm{Uni}}$
$\newcommand{\NormDist}{\mathcal{N}}$
$\newcommand{\GammaDist}{\textrm{Gamma}}$
$\newcommand{\ExpDist}{\textrm{Exp}}$
$\newcommand{\Uniform}{\textrm{Uniform}}$
$\newcommand{\Binomial}{\textrm{Binom}}$
$\newcommand{\BetaDist}{\textrm{Beta}}$
$\newcommand{\BetaFunc}{\textrm{B}}$
$\newcommand{\setify}[1]{\mathbb{#1}}$
$\newcommand{\NatSet}{\setify{N}}$
$\newcommand{\IntSet}{\setify{Z}}$
$\newcommand{\RealSet}{\setify{R}}$
$\newcommand{\CompSet}{\setify{C}}$
$\newcommand{\QuatSet}{\setify{H}}$
$\newcommand{\FieldSet}{\setify{K}}$
$\newcommand{\define}{:=}$
$\newcommand{\enifed}{=:}$
$\newcommand{\loss}{\ell}$
$\newcommand{\risk}{\textrm{R}}$
$\newcommand{\MSE}{\textrm{MSE}}$
$\newcommand{\norm}[1]{\lVert #2 \rVert}$
$\newcommand{\InnerProduct}[2]{\left( #1 , #2 \right)}$
$\newcommand{\kilogram}{\textrm{kg}}$
$\newcommand{\metre}{\textrm{m}}$
$\newcommand{\watt}{\textrm{W}}$
$\newcommand{\joule}{\textrm{J}}$
$\newcommand{\kelvin}{\textrm{K}}$
$\newcommand{\second}{\textrm{s}}$
$\newcommand{\centi}{\textrm{c}}$
$\newcommand{\bigO}{\mathcal{O}}$

## Identifying Bias of a Coin
Suppose we have a coin, which flips ‘heads’ (or 1) with probability $p \in [0, 1]$ and ‘tails’ (or 0) with probability $1 − p$. We would like to find out the value of $p$, or equivalently the bias of the coin $p - \frac{1}{2}$. We have some limited knowledge on $p$, namely $p \in [0, 1]$. Moreover, we know that the value of each observation $y \sim \Bernoulli(p)$. Hence, for the observation model $y$ given $p$ we have the law
$$\problaw{y \given p} = p^y (1 − p)^{1 − y} =
\begin{cases}
p, y = 1, \\
1 − p, y = 0.
\end{cases}$$
A sensible choice for a prior on $p$ would then be $\BetaDist(1, 1)$: 
- The support of the Beta distribution is $[0, 1]$;
- The Beta distribution turns out to be the conjugate prior of the Bernoulli distribution, which means that it is relatively straightforward to get a closed form expression for the posterior.

Suppose we have as prior $p \sim \BetaDist(\alpha, \beta)$ for some $\alpha, \beta \geq 0$. It is then possible to derive that 
$$p \mid y \sim \BetaDist(\alpha + y, \beta + 1 - y).$$
<details>
<summary><b>Proof</b></summary>
  
Suppose $p \haslaw \BetaDist(\alpha, \beta)$ for some known $\alpha, \beta > 0$ and $y \haslaw \Bernoulli(p)$. Then, by the definition of the [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution), we know that
$$\problaw{p} = \frac{p^{\alpha - 1} (1 - p)^{\beta - 1}}{\BetaFunc(\alpha, \beta)},$$
where 
$$\BetaFunc(\alpha, \beta) = \int_0^1 x^{\alpha - 1} (1 - x)^{\beta - 1} dx$$
is the Beta function. According to the Law of Total Probability, we may find that the evidence is given by
$$\problaw{y} = \int_0^1 \problaw{y \given p} \problaw{p} dp.$$
We can therefore apply Bayes' Rule to find that
$$\begin{align}
\problaw{p \given y} & = \frac{\problaw{y \given p} \problaw{p}}{\problaw{y}} = \frac{\problaw{y \given p} \problaw{p}}{\int_0^1 \problaw{y \given p} \problaw{p} dp} = \dfrac{p^y (1 - p)^{1 - y} \frac{p^{\alpha - 1} (1 - p)^{\beta - 1}}{\BetaFunc(\alpha, \beta)}}{\int_0^1 p^{y} (1 - p)^{1 - y} \frac{q^{\alpha - 1} (1 - p)^{\beta - 1}}{\BetaFunc(\alpha, \beta)} dp} 
\\
& = \dfrac{p^{\alpha + y - 1} (1 - p)^{\beta + 1 - y - 1}}{\int_0^1 p^{\alpha + y - 1} (1 - p)^{\beta + 1 - y - 1} dp} = \frac{p^{\alpha + y - 1} (1 - p)^{\beta + 1 - y - 1}}{\BetaFunc(\alpha + y, \beta + 1 - y)}.
\end{align}$$
We now recognise the probability density function (pdf) of a random variable distributed as $\BetaDist(\alpha + y, \beta + 1 - y)$, so that indeed $p \given y \haslaw \BetaDist(\alpha + y, \beta + 1 - y)$, as required.
</details>
This is all we need to come up with an iterative algorithm which allows us to estimate the parameter $p$ online as we make observations. We start with some initial prior $\problaw{p}$, and make observations $(y_1, \dots, y_{100}) \enifed y_{1:100}$. Then, we compute the posterior $\problaw{p \given y_1}$ using Bayes' Rule. Suppose we have determined the posterior $\problaw{p \given y_{1:k}}$. Then, we treat it as a prior for $p$, and update using $y_{k + 1}$.

We will now apply this in the following example. For this, we will make use of the package [Distributions.jl](https://github.com/JuliaStats/Distributions.jl), which includes most common distributions, and makes it easy to plot densities. For the calculations themselves it is actually overkill to define distributions, since we can simply update the two parameters of the Beta distribution prior directly. 

In [ ]:
using Distributions, Random, StatsPlots, StatsBase, LazySets, LaTeXStrings
Random.seed!(987654321) # Seed for reproducibility

In [ ]:
p = 0.5 # True probability of heads
N = 100 # Number of observations
prior_init = [1, 1] # Parameters of the initial prior
obs = rand(Bernoulli(p), N); # Observations

The update rule for the prior can be written in a single line:

In [ ]:
post_recursive(prior, y::Bool) = prior .+ [y, 1 - y]

Finally, we perform the inference recursively:

In [ ]:
function make_gif_recursive(prior, data::Vector{Bool}, p_true)
    post = prior
    @gif for (n, y) ∈ enumerate(data)
        post = post_recursive(post, y)
        plot(Beta(post[1], post[2]); size = (800, 800), fillalpha = 0.3, fillrange = 0, title = "Posterior after $n observations", xlabel = "p", ylabel = "density", xlim = (0, 1), legend = nothing)
        vline!([p_true])
    end 
end
make_gif_recursive(prior_init, obs, p)

As the number of observations increases, the posterior becomes narrower, and tends to centre around the true value $p = \frac{1}{2}$. The peak of the posterior distribution is the Maximum A Posteriori (MAP) estimate. It is the value of the parameter that best explains the data, while incorporating the information from the prior. This recursive approach to Bayesian inference is also called _filtering_, and we will go into more detail about it in [this notebook](Bayesics_Filtering.ipynb).

In this case it is pretty easy to directly get a _batch_ estimate, i.e. to update the prior with an entire "batch" of data at once:

In [ ]:
post_batch(prior, data::Vector{Bool}) = prior .+ [sum(data), length(data) - sum(data)]

post_final = post_batch(prior_init, obs)
plot(Beta(post_final[1], post_final[2]); size = (800, 800), fillalpha = 0.3, fillrange = 0, title = "Posterior after $(N) observations", xlabel = "p", ylabel = "density", xlim = (0, 1), legend = nothing)
vline!([p])

The posterior distribution $p(p \mid y_{1:100})$ allows us to (centred) construct credible. For instance, to construct a 95 % credible interval, we would take the 2.5 % and the 97.5 % quantile of the posterior distribution. In this example, that would give:

In [ ]:
post_dist = Beta(post_final[1], post_final[2])
left_quant = quantile(post_dist, 0.025)
right_quant = quantile(post_dist, 0.975)
left_quant, right_quant

Hence, the 95 % credible interval we have found is $[0.356, 0.548]$, which indeed contains the true value of $p$. If we use a more biased coin, we would expect to get a narrower posterior and consequently a tighter credible interval. 

In [ ]:
p = 0.9 # True probability of heads, very biased
prior_init = [1, 1] # Parameters of the initial prior
obs = rand(Bernoulli(p), N); # Observations
make_gif_recursive(prior_init, obs, p)

In [ ]:
post_final = post_batch(prior_init, obs)
post_dist = Beta(post_final[1], post_final[2])
left_quant = quantile(post_dist, 0.025)
right_quant = quantile(post_dist, 0.975)
left_quant, right_quant

Now the 95 % credible interval we have found is $[0.838, 0.951]$: it is about half as wide as the previous credible interval, but it still contains the true value of $p$.

## Multi-Armed Bandit


We could also use this filtering approach to tackle the multi-armed bandit problem. This problem is modelled on a row of $N$ slot machines, each with fixed but unknown probability of cashing out. If we consider only binary pay outs, then this system is equivalent to $N$ coins with fixed but unknown bias. 

In [ ]:
N_arms = 6
ps = rand(N_arms)
N = 100
priors_init = [[1, 1] for _ in 1:N_arms];

The goal is to maximise the gain. We have to trade off exploration, in which we flip coins that we have not tried as often yet to increase our knowledge of their success probabilities, and exploitation, in which we flip coins that we currently believe have high success probabilities. Our knowledge of the success probabilities will be encoded in the posteriors, which we can determine recursively using posterior recursive. Now we must come up with a good strategy that balances exploration and exploitation. To incorporate the exploitation aspect, we could prioritise arms whose posteriors have a high mean. To incorporate the exploration aspect, we could make use of the width of the posteriors. For instance, the 95 % credible interval will be wide if we know little about the success probability of a given coin. To combine these, we could choose arms based on the value of the top of their 95 % credible interval. This strategy is essentially the [Uniform Confidence Bound (UCB) strategy](https://en.wikipedia.org/wiki/Multi-armed_bandit). In Julia, we can select the corresponding index using the following function:

In [ ]:
function choose_arm(posts)
    dists = [Beta(post[1], post[2]) for post in posts]
    cred_int_tops = [quantile(dist, 0.975) for dist in dists]
    return argmax(cred_int_tops)[1]
end

Notably, if there are multiple arms which achieve the maximum, `argmax` will choose one at random. Then, we can simulate this strategy as follows:

In [ ]:
function plot_posts(posts, true_ps, n)
    c_arms = size(posts)[1]
    dists = [Beta(post[1], post[2]) for post in posts]
    means = mean.(dists)
    ptops = quantile.(dists, 0.975)
    pbots = quantile.(dists, 0.025)
    dist_plot = scatter(1:c_arms, means, title = "Posteriors after $n Observations", xlabel = "Arm", ylabel = "p", label = "Posterior mean", xlim = (0.5, c_arms + 0.5), ylim = (0, 1))
    scatter!(dist_plot, 1:c_arms, true_ps, label = "True p")
    for arm ∈ 1:c_arms
        ls = LineSegment([arm, pbots[arm]], [arm, ptops[arm]])
        plot!(dist_plot, ls, linecolor = "black", ylim = (0, 1))
    end
end

function explore_and_exploit_plot(priors, true_ps, sample_size)
    posts = copy(priors)
    @gif for n ∈ 1:sample_size
        arm = choose_arm(posts)
        success = rand(Bernoulli(true_ps[arm]))
        posts[arm] = post_recursive(posts[arm], success)
        plot_posts(posts, true_ps, n)
    end 
end
explore_and_exploit_plot(priors_init, ps, N)

In this example, we see that the strategy quickly finds the best arm, and then exploits it. To quantify the strategy, we compare it to an approach where the arm is chosen at random. 

In [ ]:
function explore_and_exploit(priors, true_ps, sample_size)
    posts = copy(priors)
    successes = 0
    for _ ∈ 1:sample_size
        arm = choose_arm(posts)
        success = rand(Bernoulli(true_ps[arm]))
        posts[arm] = post_recursive(posts[arm], success)
        successes += success
    end 
    return successes / sample_size
end

function random_selection(true_ps, sample_size)
    successes = 0
    for _ in 1:sample_size
        p_cur = rand(true_ps)
        successes += rand(Bernoulli(p_cur))
    end
    return successes / sample_size
end

function plot_dists_strategies_multi(priors, true_ps, sample_size, number_tests)
    dist_random = Vector{Float64}(undef, number_tests)
    dist_explore_exploit = Vector{Float64}(undef, number_tests)
    for n ∈ 1:number_tests
        dist_random[n] = random_selection(true_ps, sample_size)
        dist_explore_exploit[n] = explore_and_exploit(priors, true_ps, sample_size)
    end
    p_dists = histogram(dist_random, title = "Empirical Distributions of Average Gain after $(sample_size) Steps", normalize = :pdf, xlim = (0, 1), ylim = (0, 10), xlabel = "Average Gain", ylabel = "Density", label = "Random Selection", size = (800, 800))
    histogram!(p_dists, dist_explore_exploit, normalize = :pdf, label = "Bayesian Selection")
    vline!(p_dists, [true_ps], linecolor = "black", label = "p")
end

For this comparison, we will approximate the distribution of the average gain, i.e. the number of successes over the total number of arms pulled, after 100 goes. This approximation is done by running such a test 5000 independent times. We would expect that choosing an arm at random would give us an average gain that is roughly the average of the success probabilities, while hopefully the Bayesian approach will give a higher average gain.

In [ ]:
N_tests = 5000
plot_dists_strategies_multi(priors_init, ps, N, N_tests)

The mean of the success probabilities is $p = 0.495$, and the distribution of the average gain of the random method is roughly centred around $\bar{p} = 0.5$. On the other hand, our Bayesian approach performs much better, with an average gain that is typically almost as high as the highest success probability.